In [12]:
import pandas as pd
import numpy as np
from collections import Counter
from copy import copy
from numpy import genfromtxt
from scipy import spatial
from numpy import dot
from numpy.linalg import norm

In [29]:
df = pd.read_csv("Superpowers.csv")
netid = list(df["Cornell Tech Email (Required)"].apply(lambda x: x.split("@")[0]))
name  = list(df["Slack (Full Name) (Required)"])
name_2_netid = {n:id for n, id in zip(name, netid)}
netid_2_name = {id:n for n, id in zip(name, netid)}
index_2_name = {i:n for i, n in enumerate(name)}
name_2_index = {n:i for i, n in enumerate(name)}

In [14]:
# read data
categorical_data = genfromtxt('./cleaned_data/categorical_data.csv', delimiter=',')[1:]
bow_data = genfromtxt('./cleaned_data/bow_data.csv', delimiter=',')

data = np.concatenate((categorical_data, bow_data), axis=1)
data.shape

(350, 621)

In [15]:
# cosine similarity
cos_sim = lambda a, b: dot(a, b)/(norm(a)*norm(b))
cos_sim(data[100], data[200])

0.3326495719295578

In [16]:
# jaccard similarity
def jaccard_sim(x, y):
	intersection = np.logical_and(x, y)
	union = np.logical_or(x, y)
	similarity = intersection.sum() / float(union.sum())
	return similarity

jaccard_sim(data[100], data[200])

0.1978021978021978

In [54]:
def mehh(data, sim, n, name):
	ret = []
	for i,u1 in enumerate(data):
		u1_friends = { j:sim(u1,u2) if i!=j else 0 for j, u2 in enumerate(data) }

		# list friends based on the sim values
		u1_bff = [ (k,v) for k, v in sorted(u1_friends.items(), key=lambda item: item[1], reverse=True) ][:n]
		ret.append(u1_bff)
	
	for idx, score in ret[name_2_index[name]]:
		print("Name: {}, {}".format(index_2_name[idx], score))
	return idx


In [55]:
mehh(data=data, sim=jaccard_sim, n=15, name="Ethan Huang, CS")

Name: Evan Lin, 0.2868217054263566
Name: Zhengxing Xue, 0.2807017543859649
Name: Richa Vishwakarma, 0.2672413793103448
Name: Jarrett Coleman, CS, 0.2578125
Name: Rajesh Mudidana, 0.2537313432835821
Name: Joan La Rosa, CS, 0.24812030075187969
Name: Yang Song, 0.24489795918367346
Name: Jinxu Liu, CM, 0.24369747899159663
Name: Ridhima Sakhuja, 0.24285714285714285
Name: Crystal Tang, CS, 0.23275862068965517
Name: Oliver Zhang, 0.23076923076923078
Name: Grant Ellis, 0.22807017543859648
Name: Richard Pan, 0.22807017543859648
Name: Aman Panda, 0.22794117647058823
Name: Sal Galarza, ORIE, 0.22695035460992907


75